Prerequisite : Need to run the NLP model notebook

In [ ]:
df = pd.read_csv("../data/googleplaystore_user_reviews.csv")
df_review = pd.read_csv('../data/googleplaystore_cleaned.csv')
df = df.dropna()
review_with_ratings = pd.merge(df,df_review,on = 'App')
review_with_ratings.head()

In [ ]:
review_with_ratings['tokens'] = review_with_ratings['Review New'].apply(preprocess)

Compound Scores are added 

In [ ]:
analyzer = SentimentIntensityAnalyzer()
def sentiment_polarity(s):
    global analyzer
    polarity_scores = analyzer.polarity_scores(s)
    compound_score = polarity_scores["compound"]
    if compound_score >= 0.5:
        label = "Positive"
    elif compound_score > -0.05 and compound_score < 0.05:
        label = "Neutral"
    else:
        label = "Negative"
    return label, polarity_scores["neu"], polarity_scores["pos"], polarity_scores["neg"],compound_score

In [ ]:
df = review_with_ratings
df["Sentiment"], df["Neutral Proportion"], df["Positive Proportion"], df["Negative Proportion"],df['Compound'] =  zip(*df["Review"].apply(sentiment_polarity))
df.sample(3)

Normalize the rating to be between 1 and 5 

In [ ]:
df['Rating_pred'] = df['Compound'].apply(lambda x: (x-(-1))/(1-(-1))*(5-1)+1)

In [ ]:
np.min(df.Rating_pred)

In [ ]:
np.min(df.Compound)

In [ ]:
np.max(df.Rating_pred)

In [ ]:
np.max(df.Compound)

Calculate the average rating 

In [ ]:
Average_Rating = df.groupby('App')['Rating_pred'].mean()
df_Compare = pd.DataFrame(Average_Rating)
df_Compare.head()

In [ ]:
df_Compare = pd.merge(df_Compare,df_review, on = 'App')

In [ ]:
df_Compare.head()

In [ ]:
df_Compare = df_Compare.drop_duplicates()
df_Compare.head()

In [ ]:
df_Overall = pd.DataFrame(df_Compare['App'])
df_Overall['Rating Polarity'] = df_Compare['Rating_pred']
df_Overall['Rating Dataset'] = df_Compare['Rating']

In [ ]:
df_Overall.head()

In [ ]:
df_Overall.to_csv("../data/Compare_Rating.csv")

Compute the mse to compare between Overall Rating and the Rating predicted from polarity 

In [ ]:
mse = np.mean(np.multiply((df_Overall['Rating Polarity']-df_Overall['Rating Dataset']),
                          (df_Overall['Rating Polarity']-df_Overall['Rating Dataset'])))

In [ ]:
mse